## Paso 3 – Modelo con ajuste de hiperparámetros (`keras_tuner`)

En este paso se aplica **búsqueda automática de hiperparámetros** usando la librería [`Keras Tuner`](https://keras.io/keras_tuner/), con el objetivo de encontrar la mejor combinación de arquitectura y configuración del modelo convolucional extendido definido en el paso 2.

### Características del modelo:

- Se parte del modelo aumentado (más capas convolucionales que el base) y se parametrizan:
  - Número de capas convolucionales (`num_conv_layers`)
  - Número de filtros (`filters`) y tamaño del kernel (`kernel_size`)
  - Función de activación (`relu` o `tanh`)
  - Inicialización de pesos (`he_uniform`, `glorot_uniform`)
  - Regularización L2 (`kernel_regularizer`)
  - Inclusión opcional de `Dropout` con tasa ajustable (`dropout_rate`)
  - Número de neuronas en la capa densa (`dense_units`)

- Se emplea la técnica de `RandomSearch` para explorar combinaciones posibles de hiperparámetros en un espacio de búsqueda definido.

- El modelo se compila con el optimizador `Adam` y se entrena usando `EarlyStopping` para evitar sobreentrenamiento.

### Proceso de búsqueda:

1. Se definen los espacios de búsqueda de cada hiperparámetro.
2. Se ejecutan múltiples configuraciones (`max_trials=10`).
3. Se selecciona el modelo con mayor `val_accuracy` como el **mejor modelo encontrado**.
4. Se evalúa este modelo óptimo sobre el conjunto de test para comparar su rendimiento.

In [9]:
try:
    import keras_tuner
except ImportError:
    print("keras-tuner no encontrado. Instalando...")
    import sys
    !{sys.executable} -m pip install keras-tuner
    import keras_tuner
    print("keras-tuner instalado correctamente.")

import shutil
import os    
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, regularizers
from keras_tuner.tuners import RandomSearch
from tensorflow.keras.callbacks import EarlyStopping

from utils.dataloader import load_data_npy, DataGenerator
from utils.model_utils import save_model_and_history, save_test_results

In [2]:
# --- Carga de datos desde .npy ---
images_train, categories_train, images_val, categories_val, images_test, categories_test = load_data_npy()

# --- Generadores ---
train_gen = DataGenerator(images_train, categories_train, batch_size=16)
val_gen = DataGenerator(images_val, categories_val, shuffle=False, batch_size=16)
test_gen = DataGenerator(images_test, categories_test, shuffle=False, batch_size=16)

print(f"Train: {images_train.shape}, Validation: {images_val.shape}, Test: {images_test.shape}")

Train: (10220, 150, 150, 3), Validation: (2555, 150, 150, 3), Test: (4259, 150, 150, 3)


In [ ]:
# --- Definición del modelo para tuning ---
def build_model(hp, verbose=True):
    model = models.Sequential()
    input_shape = (150, 150, 3)
    model.add(layers.Input(shape=input_shape))
    
    if verbose:
        print(f"Forma de entrada: {input_shape}")

    num_conv_layers = hp.Int('num_conv_layers', 2, 3)
    if verbose:
        print(f"Número de capas convolucionales: {num_conv_layers}")

    for i in range(num_conv_layers):
        filters = hp.Choice(f'filters_{i}', values=[32, 64, 128])
        kernel_size = hp.Choice(f'kernel_size_{i}', values=[3, 5])
        activation = hp.Choice('activation', ['relu', 'tanh'])
        kernel_init = hp.Choice('kernel_init', ['he_uniform', 'glorot_uniform'])
        l2_reg = hp.Float('l2', 1e-5, 1e-2, sampling='LOG')

        if verbose:
            print(f"  * Capa Conv {i}: filtros={filters}, tamaño kernel={kernel_size}, activación={activation}")
            print(f"     -> Inicializador de kernel: {kernel_init}, Regularización L2: {l2_reg}")

        model.add(layers.Conv2D(filters, (kernel_size, kernel_size),
                                activation=activation,
                                kernel_initializer=kernel_init,
                                kernel_regularizer=regularizers.l2(l2_reg)))
        model.add(layers.BatchNormalization())
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Flatten())

    use_dropout = hp.Boolean('use_dropout')
    if use_dropout:
        dropout_rate = hp.Float('dropout_rate', 0.3, 0.7, step=0.1)
        if verbose:
            print(f"Dropout activado con tasa: {dropout_rate}")
        model.add(layers.Dropout(rate=dropout_rate))
    else:
        if verbose:
            print("Dropout no utilizado")

    dense_units = hp.Int('dense_units', 64, 256, step=64)
    if verbose:
        print(f"Número de neuronas en capa densa: {dense_units}")

    model.add(layers.Dense(units=dense_units, activation='relu'))
    model.add(layers.Dense(6, activation='softmax'))

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    if verbose:
        print("Modelo compilado correctamente.")
        
    return model

# --- limpieza carpeta tuner_logs/cnn_hyperparam_tuning---
shutil.rmtree('tuner_logs/cnn_hyperparam_tuning', ignore_errors=True)

# --- Búsqueda de hiperparámetros ---
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='tuner_logs',
    project_name='cnn_hyperparam_tuning'
)

early_stop = EarlyStopping(monitor='val_loss', patience=3)

# Ejecutar búsqueda
tuner.search(train_gen,
             validation_data=val_gen,
             epochs=15,
             callbacks=[early_stop])

# Obtener mejor modelo
best_model = tuner.get_best_models(num_models=1)[0]

Trial 4 Complete [02h 59m 06s]
val_accuracy: 0.8242661356925964

Best val_accuracy So Far: 0.8242661356925964
Total elapsed time: 09h 32m 10s

Search: Running Trial #5

Value             |Best Value So Far |Hyperparameter
2                 |3                 |num_conv_layers
64                |32                |filters_0
3                 |3                 |kernel_size_0
tanh              |tanh              |activation
glorot_uniform    |glorot_uniform    |kernel_init
7.1292e-05        |0.00010081        |l2
64                |128               |filters_1
3                 |3                 |kernel_size_1
False             |True              |use_dropout
256               |64                |dense_units
128               |64                |filters_2
3                 |3                 |kernel_size_2
0.6               |0.4               |dropout_rate

Forma de entrada: (150, 150, 3)
Número de capas convolucionales: 2
  * Capa Conv 0: filtros=64, tamaño kernel=3, activación=tanh
   

In [ ]:
# Reentrena el mejor modelo con historial completo
history = best_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15,
    callbacks=[EarlyStopping(monitor='val_loss', patience=5)]
)

# Evaluación final
test_loss, test_acc = best_model.evaluate(test_gen)
print(f"\nTest Accuracy: {test_acc:.4f}  |  Test Loss: {test_loss:.4f}")

# Ahora sí puedes guardarlo
save_model_and_history(best_model, history, model_path='tuned_model')
save_test_results('tuned_model', test_loss, test_acc)